# 순환 신경망

### 1. 라이브러리 설치 및 예시 문장 제작

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

n_hidden = 35
lr = 0.01
epochs = 100

In [2]:
string = "hello pytorch. how long can a rnn cell remember? show me your limit!"
chars =  "abcdefghijklmnopqrstuvwxyz ?!.,:;01"

char_list = [i for i in chars]
n_letters = len(char_list)

### 2. 필요한 함수 선언

In [5]:
# 문자열을 one-hot vector의 stack으로 만드는 함수

def string_to_onehot(string):
    start = np.zeros(shape=n_letters, dtype=int) # start token
    end = np.zeros(shape=n_letters,dtype=int)    # end token
    start[-2] = 1
    end[-1] = 1
    for i in string:
        idx = char_list.index(i)
        zero = np.zeros(shape=n_letters, dtype=int)
        zero[idx] = 1
        start = np.vstack([start,zero])
    
    output = np.vstack([start, end])
    return output

In [7]:
# one-hot vector를 문자로 바꿔주는 함수

def onehot_to_word(onehot_1):
    onehot = torch.Tensor.numpy(onehot_1) # Tensor를 입력받아 numpy로 변경
    return char_list[onehot.argmax()]     # one-hot 벡터의 최대값(-1) 위치 index로 문자 탐색


### 3. 모델 구축 및 평가

In [14]:
# 순환 신경망

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.act_fn = nn.Tanh()
    
    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1) # 입력과 hidden state를 cat으로 결합
        hidden = self.act_fn(self.i2h(combined))
        output = self.i2o(combined)
        return output, hidden # 결합한 값을 i2h및 i2o에 통과시켜 hidden state를 업데이트하고, 결과값을 계산
    
    def init_hidden(self):
        return torch.zeros(1, self.hidden_size) # 입력이 없을 때(t=0) hidden_state 초기화
    
rnn = RNN(n_letters, n_hidden, n_letters)

In [15]:
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr = lr)

In [16]:
# train

one_hot = torch.from_numpy(string_to_onehot(string)).type_as(torch.FloatTensor())

for i in range(epochs):
    optimizer.zero_grad()
    hidden = rnn.init_hidden()
    
    total_loss = 0
    for j in range(one_hot.size()[0]-1):
        # 입력은 앞의 글자
        input_ = one_hot[j:j+1,:]
        target = one_hot[j+1]
        output, hidden = rnn.forward(input_, hidden)
        
        loss = loss_func(output.view(-1), target.view(-1))
        total_loss += loss
    
    total_loss.backward()
    optimizer.step()
    
    if i % 10 == 0:
        print(total_loss)

tensor(2.6785, grad_fn=<AddBackward0>)
tensor(1.1156, grad_fn=<AddBackward0>)
tensor(0.6694, grad_fn=<AddBackward0>)
tensor(0.4131, grad_fn=<AddBackward0>)
tensor(0.3118, grad_fn=<AddBackward0>)
tensor(0.2210, grad_fn=<AddBackward0>)
tensor(0.1637, grad_fn=<AddBackward0>)
tensor(0.1322, grad_fn=<AddBackward0>)
tensor(0.1064, grad_fn=<AddBackward0>)
tensor(0.0891, grad_fn=<AddBackward0>)


In [18]:
# test

start = torch.zeros(1, n_letters)
start[:,-2] = 1

with torch.no_grad():
    hidden = rnn.init_hidden()
    input_ = start
    output_string = ""
    
    for i in range(len(string)):
        output, hidden = rnn.forward(input_, hidden)
        output_string += onehot_to_word(output.data)
        input_ = output

print(output_string)

hello pytorcr  hoowlm a me me r l mem m mrimilimlilelimiml1limimimli
